---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Date Extract and Sort from Medical Text

This code reads medical text data and uses regex to extract dates and order them. Each line of the text file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of a variety formats. The code returns a pandas Series in chronological order of the original Series' indices.

In [2]:
import pandas as pd
import re
    
doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

In [15]:
def date_sorter():
    
    # Your code here

    #---------------------------------------------------------------------------------------
    # Function to add Century
    #---------------------------------------------------------------------------------------
    
    def add_century(my_date):
        if re.search('[-/]\d{2}\Z', my_date):  
            # Extract position of beginning of pattern
            pos = re.search('\d{2}\Z', my_date).start()
            return my_date[0:pos] + '19' + my_date[pos:]
        else:
            # if already has century, return the same name
            return my_date

    #---------------------------------------------------------------------------------------
    # Setup
    #---------------------------------------------------------------------------------------
    
    s = pd.Series(doc, name="txt")
    df = s.to_frame()
    df['original_index'] = df.index

    #---------------------------------------------------------------------------------------
    # Regex
    #---------------------------------------------------------------------------------------

    # 1) 04/20/2009; 04/20/09; 4/20/09; 4/3/09 : Line 0~124 (121 entries)
    df1 = df[:125]
    df1 = df1['txt'].str.extractall(r'(?P<date>[01]?\d{1}[/-][0123]*\d{1}[/-][12]*[90]*[78901]\d{1})')
    df1['date'] = df1['date'].apply(add_century)
    # Format= 01/01/2008 or 4-13-1982
    df1['date'] = pd.to_datetime(df1['date'])

    # 2) Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009; : Line 194~227 (34 entries)
    df2 = df[194:228]
    df2 = df2['txt'].str.extractall(r'(?P<date>(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[.]*[a-z]*[.]*[ \-]+\d{1,2}[\-, ]+\d{4})', re.IGNORECASE)
    df2.date.replace(to_replace='\.', regex=True, value = '', inplace=True) 
    df2.date.replace(regex={'anuary':'an', 'anaury':'an', 'ebruary':'eb', 'arch':'ar', 'pril':'pr',
                            'une':'un', 'uly':'ul', 'ugust': 'ug', 'eptember':'ep', 
                            'ctober':'ct', 'ovember':'ov', 'ecember':'ec', 'ecemeber':'ec'}, inplace=True ) 
    df2.date.replace(to_replace='\,', regex=True, value = '', inplace=True) 
    # Format=Apr 11 1990
    df2['date'] = pd.to_datetime(df2['date'], format='%b %d %Y')

    # 3) 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009 : Line 125~193 (69 entries)
    df3 = df[125:194]
    #df3.iloc[39]
    df3 = df3['txt'].str.extractall(r'(?P<date>\d{1,2} (?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[.]*[a-z]*[.]* +\d{4})', re.IGNORECASE)
    df3.date.replace(regex={'anuary':'an', 'anaury':'an', 'ebruary':'eb', 'arch':'ar', 'pril':'pr',
                            'une':'un', 'uly':'ul', 'ugust': 'ug', 'eptember':'ep', 
                            'ctober':'ct', 'ovember':'ov', 'ecember':'ec', 'ecemeber':'ec'}, inplace=True ) 
    # Format=24 Jan 2001
    df3['date'] = pd.to_datetime(df3['date'], format='%d %b %Y')

    # 4) Feb 2009; Sep 2009; Oct 2010 : Line 228~342 (115 entries)
    df4 = df[228:343]
    df4 = df4['txt'].str.extractall(r'(?P<date>(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[.]*[a-z]*[,]* \d{4})', re.IGNORECASE)
    # Data cleanup:
    df4.date.replace(to_replace=', ', regex=True, value = ' ', inplace=True) 
    df4.date.replace(to_replace=' ', regex=True, value = ' 1, ', inplace=True) 
    df4.date.replace(regex={'anuary':'an', 'anaury':'an', 'ebruary':'eb', 'arch':'ar', 'pril':'pr',
                            'une':'un', 'uly':'ul', 'ugust': 'ug', 'eptember':'ep', 
                            'ctober':'ct', 'ovember':'ov', 'ecember':'ec', 'ecemeber':'ec'}, inplace=True ) 
    # Format=Sep 1, 1985
    df4['date'] = pd.to_datetime(df4['date'], format='%b %d, %Y')

    # 5) 6/2008; 12/2009: Line 343~454 (112 entries)
    df5 = df[343:455]
    df5 = df5['txt'].str.extractall(r'(?P<date>\d{1,2}/\d{4})')
    df5.date.replace(to_replace='/', regex=True, value = '/01/', inplace=True) 
    # Format= 6/01/2008
    df5['date'] = pd.to_datetime(df5['date'], format='%m/%d/%Y')

    # 6) 2009; 2010: Line 455~499 (45 entries)
    df6 = df[455:500]
    df6 = df6['txt'].str.extractall(r'(?P<date>\d{4})')
    df6['date'] = '01/01/' + df6['date']
    # Format= 01/01/2008
    df6['date'] = pd.to_datetime(df6['date'], format='%m/%d/%Y')

    #---------------------------------------------------------------------------------------
    # MERGE and SORT
    #---------------------------------------------------------------------------------------

    df_all = df1.append(df3, ignore_index=True)
    df_all = df_all.append(df2, ignore_index=True)
    df_all = df_all.append(df4, ignore_index=True)
    df_all = df_all.append(df5, ignore_index=True)
    df_all = df_all.append(df6, ignore_index=True)

    df_all = df_all.sort_values("date")
    df_all.reset_index(inplace=True)

    #---------------------------------------------------------------------------------------
    # DEBUG
    #---------------------------------------------------------------------------------------    
    #df_all.iloc[0:100]
    #df1.iloc[24][0]
    #df6.head(45)
    #len(df6)
    #---------------------------------------------------------------------------------------
    # RETURN
    #---------------------------------------------------------------------------------------
    
    return df_all["index"]

date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Name: index, Length: 500, dtype: int64